In [16]:
import open3d as o3d
import numpy as np
import os
import copy

In [5]:
def pcd_templates(temp_path,body_name):
    geo=temp_pipe_path+"\\"+body_name
    temp_stl=o3d.io.read_triangle_mesh(geo)
    temp_pcd=temp_stl.sample_points_uniformly(number_of_points=int(temp_stl.get_surface_area()*0.1))
    temp_pcd.points = o3d.utility.Vector3dVector(np.asarray(temp_pcd.points)-temp_pcd.get_center())
    return temp_pcd

def get_profile(pcd):
    obb=pcd.get_oriented_bounding_box()
    pp=np.asarray(obb.get_box_points())
    ND=np.sort(np.linalg.norm(pp-pp[0],axis=1))[-4]
    NV=(pp-pp[0])[np.where(np.linalg.norm(pp-pp[0],axis=1)==ND)]
    nV=NV/np.linalg.norm(NV)
    PP=np.sort(np.linalg.norm(pp-pp[0],axis=1))[:4]
    PP=(pp)[np.where(np.sum(np.linalg.norm(pp-pp[0],axis=1)==PP.reshape(-1,1),axis=0).astype(bool))]
    P1=PP+nV*5
    P2=PP+nV*25
    vobb=o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(np.vstack((P1,P2))))
    prof=pcd.crop(vobb)
    prof.points = o3d.utility.Vector3dVector(np.asarray(prof.points)-prof.get_center())
    return prof,nV

v_pcd_templates=np.vectorize(pcd_templates,signature='(),()->()')
v_get_profile=np.vectorize(get_profile,signature='()->(),(k)')

def get_length(tar_pcd):
    obb=tar_pcd.get_oriented_bounding_box()
    pp=np.asarray(obb.get_box_points())
    return np.sort(np.linalg.norm(pp-pp[0],axis=1))[-4]

def gen_temp(prof,nV,tar_pcd):
    L=get_length(tar_pcd)
    transd=(np.arange(0,int(L/20))*20).reshape(1,-1).T*nV
    transtemps=np.vectorize(pyfunc=(lambda x: (np.asarray(prof.points)+x)),
                        signature="(k)->(j,k)")(transd).reshape(-1,3)
    temppcd = o3d.geometry.PointCloud()
    temppcd.points = o3d.utility.Vector3dVector(transtemps-np.mean(transtemps,axis=0))
    return temppcd

v_gen_temp=np.vectorize(gen_temp,signature='(),(k),()->()')

In [6]:
#Carga de templates
temppath="D:\Documentos\INNOVATE\GH\proyectox\RANSAC\Templates"
temp_pipe_path=temppath+"\Pipes"
temp_beams_path=temppath+"\Beams"
names=np.array(os.listdir(temp_pipe_path))
templates=v_pcd_templates(temp_pipe_path,names)
cpcds,nVs=v_get_profile(templates)

In [7]:
#carga de Base de datos
temppath="D:\Documentos\INNOVATE\Base_de_datos\Aceptado"
names=np.array(os.listdir(temppath))
tar_pcd_dir=temppath+"\\"+names[4]
tar_pcd=o3d.io.read_point_cloud(tar_pcd_dir)
tar_pcd.points=o3d.utility.Vector3dVector(np.asarray(tar_pcd.points)-tar_pcd.get_center())

print(get_length(tar_pcd))
o3d.visualization.draw_geometries([tar_pcd,gen_temp(cpcds[4],nVs[4],tar_pcd)])

2398.569637482869


In [8]:
pcd_temp=v_gen_temp(cpcds,nVs,tar_pcd)

In [9]:
o3d.visualization.draw_geometries([pcd_temp[0],pcd_temp[1],pcd_temp[5]])

In [14]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

def prepare_dataset(voxel_size,source,target):
    print(":: Load two point clouds and disturb initial pose.")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down,
        target_down,
        source_fpfh,
        target_fpfh,
        True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3,
        [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [19]:
source_down, target_down, source_fpfh, target_fpfh=prepare_dataset(0.05,pcd_temp[0],tar_pcd)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [ ]:
result_ransac = execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, 0.05)